# 제 2유형_연습하기_당뇨진척정도(회귀)

✅ 데이터 분석 순서             
1. 라이브러리 및 데이터 확인
2. 데이터 탐색 (EDA)
3. 데이터 전처리 및 분리 (train data → train / validation data)
4. 모델링 및 성능 평가
5. 예측값 제출

### ✅ 1. 라이브러리 및 데이터 확인

In [1]:
import pandas as pd 
import numpy as np

In [6]:
############### 실기환경 복사 영역 ############### import pandas as pd
import numpy as np
# 실기 시험 데이터셋으로 셋팅하기 (수정금지)
from sklearn.datasets import load_diabetes
# diabetes 데이터셋 로드
diabetes = load_diabetes()
x = pd.DataFrame(diabetes.data, columns=diabetes.feature_names) 
y = pd.DataFrame(diabetes.target)

# 실기 시험 데이터셋으로 셋팅하기 (수정금지)
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=2023)

x_test = pd.DataFrame(x_test.reset_index())
x_train = pd.DataFrame(x_train.reset_index())
y_train = pd.DataFrame(y_train.reset_index())

x_test.rename(columns={'index':'cust_id'}, inplace=True) 
x_train.rename(columns={'index':'cust_id'}, inplace=True) 
y_train.columns = ['cust_id', 'target']


### 참고사항 ###
# y_test 는 실기 문제상에 주어지지 않음
# ★Tip : X를 대문자로 쓰지말고 소문자 x로 쓰세요. 시험에서 실수하기 쉽습니다.(문제풀기 전에 소문자로 변경!) 
# # (참고 : 보통 X는 2차원 배열(행렬)이기 때문에 대문자로 쓰고, y는 1차원 배열(벡터)이기 때문에 소문자로 씀)
# (~23년 10월말) 실기시험 데이터 형식 (실제 시험장에서는 다를 수 있으니 반드시 체크) 
# # X_test = pd.read_csv("data/X_test.csv")
# X_train = pd.read_csv("data/X_train.csv")
# y_train = pd.read_csv("data/y_train.csv")

# ★(23년 10월말~) 기준으로 체험환경에서 제공되는 데이터셋이 조금 변경되었습니다. 
# train = pd.read_csv("data/customer_train.csv")
# test = pd.read_csv("data/customer_test.csv")          # 제출할 때만 쓰는 데이터. 분리하지마!
# (x_train과 y_train, x_test를 별도로 할당해주셔야 합니다.)
# 주어진 train 데이터만 train / validation 데이터셋으로 분리해서 예측 모델 만들기
# pandas 라이브러리이므로 iloc나 loc를 사용하여 컬럼 확인해서 넣기
# x_train, x_val, y_train, y_val = (train[:, 타겟데이터 제외한 모든 열], train[:, 타겟데이터 열], test_size=0.2, random_state=2023)

당뇨병 환자의 질병 진행 정도를 예측해보자.          
- 데이터의 결측치, 이상치, 변수들에 대해 전처리하고         
- 회귀모델을 사용하여 Rsq, MSE 값을 산출하시오.         
- 제출은 cust_id, target 변수를 가진 dataframe 형태로 제출하시오.

In [7]:
# 데이터 설명
print(diabetes.DESCR)

# target : 1년이 지난 후 당뇨가 얼마나 진척되었는지 알려주는 지표(수치)

.. _diabetes_dataset:

Diabetes dataset
----------------

Ten baseline variables, age, sex, body mass index, average blood
pressure, and six blood serum measurements were obtained for each of n =
442 diabetes patients, as well as the response of interest, a
quantitative measure of disease progression one year after baseline.

**Data Set Characteristics:**

  :Number of Instances: 442

  :Number of Attributes: First 10 columns are numeric predictive values

  :Target: Column 11 is a quantitative measure of disease progression one year after baseline

  :Attribute Information:
      - age     age in years
      - sex
      - bmi     body mass index
      - bp      average blood pressure
      - s1      tc, total serum cholesterol
      - s2      ldl, low-density lipoproteins
      - s3      hdl, high-density lipoproteins
      - s4      tch, total cholesterol / HDL
      - s5      ltg, possibly log of serum triglycerides level
      - s6      glu, blood sugar level

Note: Each of these 1

### ✅ 2. 데이터 탐색(EDA)

In [8]:
# 데이터의 행 / 열 확인
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)

(353, 11)
(89, 11)
(353, 2)


In [9]:
# 초기 데이터 확인
print(x_train.head(3))
print(x_test.head(3))
print(y_train.head(3))

   cust_id       age       sex       bmi        bp        s1        s2  \
0        4  0.005383 -0.044642 -0.036385  0.021872  0.003935  0.015596   
1      318  0.088931 -0.044642  0.006728  0.025315  0.030078  0.008707   
2      301 -0.001882  0.050680 -0.024529  0.052858  0.027326  0.030001   

         s3        s4        s5        s6  
0  0.008142 -0.002592 -0.031988 -0.046641  
1  0.063367 -0.039493  0.009434  0.032059  
2  0.030232 -0.002592 -0.021395  0.036201  
   cust_id       age       sex       bmi        bp        s1        s2  \
0      280  0.009016  0.050680  0.018584  0.039087  0.017694  0.010586   
1      412  0.074401 -0.044642  0.085408  0.063187  0.014942  0.013091   
2       68  0.038076  0.050680 -0.029918 -0.040099 -0.033216 -0.024174   

         s3        s4        s5        s6  
0  0.019187 -0.002592  0.016307 -0.017646  
1  0.015505 -0.002592  0.006207  0.085907  
2 -0.010266 -0.002592 -0.012909  0.003064  
   cust_id  target
0        4   135.0
1      318   109

In [11]:
# 변수명과 데이터 타입이 매칭이 되는지, 결측치가 있는지 확인
# 특히 target data의 경우 target 컬럼만 사용이 되어야 하므로 다른 컬럼이 있는지 잘 확인해봐야 함
print(x_train.info())
print(x_test.info())
print(y_train.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 353 entries, 0 to 352
Data columns (total 11 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   cust_id  353 non-null    int64  
 1   age      353 non-null    float64
 2   sex      353 non-null    float64
 3   bmi      353 non-null    float64
 4   bp       353 non-null    float64
 5   s1       353 non-null    float64
 6   s2       353 non-null    float64
 7   s3       353 non-null    float64
 8   s4       353 non-null    float64
 9   s5       353 non-null    float64
 10  s6       353 non-null    float64
dtypes: float64(10), int64(1)
memory usage: 30.5 KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 89 entries, 0 to 88
Data columns (total 11 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   cust_id  89 non-null     int64  
 1   age      89 non-null     float64
 2   sex      89 non-null     float64
 3   bmi      89 non-null     float64
 4   bp      

실제 sex 컬럼의 경우 범주형 데이터이지만 scaling 되어 입력된 듯.

In [12]:
# x_train과 x_test 데이터의 기초통계량 비교 (분포가 비슷한지 꼭 확인!)
print(x_train.describe())
print(x_test.describe())
print(y_train.describe())

          cust_id         age         sex         bmi          bp          s1  \
count  353.000000  353.000000  353.000000  353.000000  353.000000  353.000000   
mean   212.634561    0.000804    0.000724    0.000640   -0.000326    0.001179   
std    126.668903    0.047617    0.047673    0.048141    0.046585    0.047891   
min      0.000000   -0.107226   -0.044642   -0.084886   -0.112399   -0.126781   
25%    105.000000   -0.038207   -0.044642   -0.035307   -0.033213   -0.033216   
50%    210.000000    0.005383   -0.044642   -0.006206   -0.005670   -0.002945   
75%    322.000000    0.038076    0.050680    0.030440    0.032201    0.027326   
max    441.000000    0.110727    0.050680    0.170555    0.125158    0.153914   

               s2          s3          s4          s5          s6  
count  353.000000  353.000000  353.000000  353.000000  353.000000  
mean     0.001110   -0.000452    0.000901    0.001446    0.000589  
std      0.048248    0.048600    0.048045    0.047160    0.048122 

In [16]:
# y 데이터도 확인
print(y_train.head())
print(y_train.describe().T)

   cust_id  target
0        4   135.0
1      318   109.0
2      301    65.0
3      189    79.0
4      288    80.0
         count        mean         std   min    25%    50%    75%    max
cust_id  353.0  212.634561  126.668903   0.0  105.0  210.0  322.0  441.0
target   353.0  152.943343   75.324692  37.0   90.0  141.0  208.0  346.0


### ✅ 3. 데이터 전처리 및 분리
#### 1) 결측치, 2) 이상치, 3) 변수 처리하기

In [17]:
# 결측치 확인
print(x_train.isnull().sum())
print(x_test.isnull().sum())
print(y_train.isnull().sum())

cust_id    0
age        0
sex        0
bmi        0
bp         0
s1         0
s2         0
s3         0
s4         0
s5         0
s6         0
dtype: int64
cust_id    0
age        0
sex        0
bmi        0
bp         0
s1         0
s2         0
s3         0
s4         0
s5         0
s6         0
dtype: int64
cust_id    0
target     0
dtype: int64


In [ ]:
# 결측치 제거
# df = df.dropna()
# print(df)

# 행 기준 삭제
# print(df.dropna().shape)

# ★주의사항
# x_train의 행을 제거해야 하는 경우, 그에 해당하는 y_train 행도 제거해야 함.
# ⭐️ train data 전처리를 완료한 뒤에 train & valid data 분리하기!

In [ ]:
# 결측치 대체 (평균, 중앙값, 최빈값) : 데이터 분포에 따라 어떤 값으로 대체하느냐가 다르긴 하지만 보통 중앙값을 많이 사용. (국단값의 영향을 덜 받음)
# 연속형 변수 : 중앙값, 평균
    # df['변수명'].median()
    # df['변수명'].mean()
# 범주형 변수 : 최빈값
    # df['변수명'].mode()

# 결측치 대체
# df['변수명'] = df['변수명'].fillna(대체할 값)

# ⭐️ 주의사항 : train 데이터의 중앙값/평균값/최빈값 등으로 test 데이터의 결측치도 변경해줘야 함
# → 전처리 완료 후 데이터 결측치 대체까지 하고 데이터 분리하자

In [ ]:
# 이상치 확인 및 대체 : describe()로 확인
# df['변수명'] = np.where(df['변수명'] >= 5, 대체할 값, df['변수명'])

### ✅ 1. 라이브러리 및 데이터 확인

### ✅ 1. 라이브러리 및 데이터 확인